## Miniproject: Comparador precios supermercados


- 500 g Carne picada de cerdo y ternera
- 100 gTomate frito
- 100g Harina
- 100g Mantequilla
- 1l Leche
- 1 Cebolla
- 1 Ajo
- 16 laminas Pasta para canelones - equivale a 250gramos
- 250g Queso rallado
- 60 ml Aceite de oliva
- 15 gramos de Sal



In [92]:
dict_receta = {'preparado mixto carne' : 0.5, 'tomate' : 0.1, 'harina': 0.1, 'mantequilla':0.1, 'leche': 1, 'cebolla': 0.2, 'ajo':0.06, 'pasta': 0.25, 'quesos': 0.250, 'aceite': 0.06, 'sal': 0.015}

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
import re
import numpy as np

### Listas de las urls para cada supermercado

In [2]:
ingredientes_bonpreu = ['https://www.compraonline.bonpreuesclat.cat/products?sortBy=favorite&sublocationId=b4098292-6a2a-428f-ace6-076a19878890',
                        'https://www.compraonline.bonpreuesclat.cat/products?sortBy=favorite&sublocationId=22a85cdc-fce2-4818-adad-9bd150c0a38d',
                        'https://www.compraonline.bonpreuesclat.cat/products?sortBy=favorite&sublocationId=8d28608c-a0b0-4fe5-8104-e906f732365c',
                        'https://www.compraonline.bonpreuesclat.cat/products?sortBy=favorite&sublocationId=71f75f2f-0352-4035-8e6e-329727bb2167',
                        'https://www.compraonline.bonpreuesclat.cat/products?sortBy=favorite&sublocationId=28fde871-8fb9-4f23-a386-ae6c8d64a149',
                        'https://www.compraonline.bonpreuesclat.cat/products?sortBy=favorite&sublocationId=a8be0b30-ea43-431a-af01-5defaa0072b7',
                        'https://www.compraonline.bonpreuesclat.cat/products?sortBy=favorite&sublocationId=1a5c3d3d-7f00-48ef-bb8e-ff26aad9e89c',
                        'https://www.compraonline.bonpreuesclat.cat/products?sortBy=favorite&sublocationId=7f1b40c8-35e4-44b5-b07a-eb878b788c17',
                        'https://www.compraonline.bonpreuesclat.cat/products?sortBy=favorite&sublocationId=7d568dd6-2e26-4f25-8149-c67f3f0e33a5',
                        'https://www.compraonline.bonpreuesclat.cat/products?sortBy=favorite&sublocationId=7a425422-8ca4-45f1-8294-29fa0deafcef']

In [3]:
ingredientes_bonarea = ['https://www.bonarea-online.com/categorias/ternera/13_300_010_070', 
                        'https://www.bonarea-online.com/categorias/verdura/13_300_060_020', 
                        'https://www.bonarea-online.com/categorias/aceite-de-oliva/13_300_110_020_010',
                        'https://www.bonarea-online.com/categorias/sal/13_300_110_030', 
                        'https://www.bonarea-online.com/categorias/harina/13_300_140_020_010',
                        'https://www.bonarea-online.com/categorias/pasta/13_300_140_030',
                        'https://www.bonarea-online.com/categorias/lacteos-y-derivados/13_300_080_010',
                        'https://www.bonarea-online.com/categorias/nata-mantequilla-y-margarina/13_300_080_030',
                        'https://www.bonarea-online.com/categorias/base-de-tomate-y-ketchup/13_300_150_020']

In [4]:
ingredientes_dia = ['https://www.dia.es/compra-online/despensa/salsas/tomate/cf', 
                    'https://www.dia.es/compra-online/despensa/lacteos-y-huevos/leche/cf',
                    'https://www.dia.es/compra-online/frescos/carne/mixto/cf', 
                    'https://www.dia.es/compra-online/despensa/pastas-harinas-y-masas/pastas/cf/tipo+canelones-y-lasana',
                    'https://www.dia.es/compra-online/despensa/pastas-harinas-y-masas/harinas-y-levaduras/cf/producto+harina',
                    'https://www.dia.es/compra-online/despensa/aceites-vinagres-y-alinos/aceites/cf/producto+aceite-de-oliva',
                    'https://www.dia.es/compra-online/frescos/charcuteria-y-quesos/quesos/cf/corte+rallado',
                    'https://www.dia.es/compra-online/despensa/lacteos-y-huevos/mantequilla-y-margarina/cf/producto+mantequilla',
                    'https://www.dia.es/compra-online/frescos/verduras-y-hortalizas/ajos-cebollas-y-puerros/cf',
                    'https://www.dia.es/compra-online/despensa/sal-y-especias/cf/producto+sal']

### Web scrapping

In [34]:
def scrapping_dia (i): 
        html = requests.get(i) 
        soup = BeautifulSoup(html.content, "html.parser")
        
        precio_unidad = [float(i.getText().strip().replace(',','.')[0:4]) for i in soup.find_all('p',{'class':'price'})]
        caracteristicas = [i.getText().strip() for i in soup.find_all('span',{'class':'details'})]
        precio_kg_l = [float(i.getText().strip().replace(',','.')[0:4]) for i in soup.find_all('p',{'class':'pricePerKilogram'})]
        link = ['https://www.dia.es/'+ i.find('a').get('href') for i in soup.find_all('div',{'class':'product-list__item'})]
        cantidad_producto = [round(precio_unidad[i]/precio_kg_l[i],1) for i in range(len(precio_unidad))]
    
        dict_ = {'precio_unidad':precio_unidad, 'caracteristicas':caracteristicas, 'precio_kg_l':precio_kg_l,'cantidad_producto': cantidad_producto, 'link': link}
    
        df = pd.DataFrame (dict_)
        
        return df

In [48]:
def scrapping_bonpreu (url):
    html = requests.get(url) 
    soup = BeautifulSoup(html.content, "html.parser")
    
    caracteristicas = [i.find('a').getText().strip() for i in soup.find_all('h3',{'class':'text__Text-sc-6l1yjp-0 iWlLMY'})]
    precio_unidad = [float(i.getText().strip().replace(',','.')[0:3]) for i in soup.find_all('strong',{'class':'base__Price-sc-1mnb0pd-29 sc-jRQAMF iDLLj hFlPaZ'})]
    
    precio1_pre = [i.getText().strip().replace(',','.') for i in soup.find_all('span',{'class':'text__Text-sc-6l1yjp-0 standard-promotion__PromotionIntentText-sc-1vpsrpe-2 fop__PricePerText-sc-1e1rsqo-3 dLNLFE jVJmKC eJshgd'})]
    precio_kg_l = [float(re.search(r'\d+\.\d+', i).group()) for i in precio1_pre]
    link = ['https://www.compraonline.bonpreuesclat.cat/' + i.find('a').get('href') for i in soup.find_all('h3',{'class':'text__Text-sc-6l1yjp-0 iWlLMY'})]
    cantidad_producto = [round (precio_unidad[i]/precio_kg_l[i],3) for i in range(len(precio_unidad))]

    dict_ = {'precio_unidad':precio_unidad, 'caracteristicas':caracteristicas, 'precio_kg_l':precio_kg_l, 'cantidad_producto': cantidad_producto, 'link':link}
    
    
    return pd.DataFrame(dict_)

In [7]:
def scrapping_bonarea (url): 
    html = requests.get(url) 
    soup = BeautifulSoup(html.content, "html.parser")
    
    precios_pre = [i.getText().split() for i in soup.find_all('div',{'class':'price'})]
    precio_unidad = [float(sublista[0].replace(',','.')) for sublista in precios_pre]
    precio_kg_l = [float(sublista[2].replace(',','.').replace('(','')) for sublista in precios_pre]
    caracteristicas = [i.getText() for i in soup.find_all('div',{'class':'text'})]
    cantidad_producto = [round (precio_unidad[i]/precio_kg_l[i],1) for i in range(len(precio_unidad))]
    
    dict_ = {'precio_unidad':precio_unidad, 'caracteristicas':caracteristicas, 'precio_kg_l':precio_kg_l, 'cantidad_producto': cantidad_producto}
    
    return pd.DataFrame (dict_)

In [8]:
def df_supermercados (list_ingredientes):
   
    df_super = pd.DataFrame () # creo un data frame vacio donde concatenaré para cada producto

    for i in list_ingredientes: #loop de la lista ingredientes para cada super, cada link es la categoria del ingrediente
            
        if 'bonpreu' in i:
            df_producto = scrapping_bonpreu(i)
            df_super = pd.concat([df_producto, df_super], ignore_index=True)
            
    
        if 'dia' in i: 
            df_producto = scrapping_dia(i)
            df_super = pd.concat([df_producto, df_super], ignore_index=True)
        
        if 'bonarea' in i:
            df_producto = scrapping_bonarea(i)
            df_super = pd.concat([df_producto, df_super], ignore_index=True)
            
            

    return df_super

### Obtención DataFrames

#### Bonarea

In [9]:
df_bonarea_pre = df_supermercados(ingredientes_bonarea)

In [10]:
def scrapping_quesos_bonarea(url): 
    
    html = requests.get('https://www.bonarea-online.com/categorias/rallados/13_300_050_050?order=3') 
    soup = BeautifulSoup(html.content, "html.parser")
    
    precios = [i.getText().split() for i in soup.find_all('div',{'class':'price'})][0:10]
    
    
    precio_unidad = [float(sublista[0].replace(',','.').replace('€','')) for sublista in precios]
    precio_kg_l = [float(sublista[2].replace(',','.').replace('(','')) for sublista in precios]
    caracteristicas = [i.getText() for i in soup.find_all('div',{'class':'text'})][0:10]
    cantidad_producto = [round (precio_unidad[i]/precio_kg_l[i],1) for i in range(len(precio_unidad))]

    
    dict_ = {'precio_unidad':precio_unidad, 'caracteristicas':caracteristicas, 'precio_kg_l':precio_kg_l, 'cantidad_producto': cantidad_producto}
    
    df_bonarea_pre = pd.DataFrame (dict_)
    df_bonarea = pd.concat([df_bonarea_pre, df_bonarea_quesos], ignore_index=True)
    
    return df_bonarea

In [11]:
df_bonarea_quesos = scrapping_quesos_bonarea ('https://www.bonarea-online.com/categorias/rallados/13_300_050_050?order=3')
df_bonarea = pd.concat([df_bonarea_pre, df_bonarea_quesos], ignore_index=True)
df_bonarea

,precio_unidad,caracteristicas,precio_kg_l,cantidad_producto
0,0.55,Tomate frito bric400 g,1.38,0.4
1,0.75,Ketchup dosificador300 g,2.50,0.3
2,1.34,Tomate frito con cebolla570 g,2.35,0.6
3,2.05,Ketchup Heinz pet342 g,5.99,0.3
4,1.85,Ketchup Prima dosificador290 g,6.38,0.3
...,...,...,...,...
266,1.80,Queso rallado 4 quesos200 g,9.00,0.2
267,1.39,Queso rallado Grana Padanoaprox. 100 g,13.90,0.1
268,0.75,Queso en polvo Hochland parma estilo italiano50 g,15.00,0.1
269,2.36,Queso rallado El Caserio cuatro quesos140 g,16.86,0.1


In [12]:
def tipo_producto_bonarea (i): 
    valor = 0
        
    if 'Tomate frito' in i:
        valor ='tomate'
    elif 'Sal' in i: 
        valor = 'sal'
    elif 'Aceite' in i or 'Oli' in i:
        valor = 'aceite' 
    elif 'Leche' in i:
        valor = 'leche'
    elif 'mixta' in i or 'Carne picada' in i: 
        valor = 'preparado mixto carne'
    elif 'Canelones' in i:
        valor = 'pasta'
    elif 'Mantequilla' in i:
        valor = 'mantequilla'
    elif 'Harina' in i: #columna caracteristicas
        valor ='harina'
    elif 'Queso' in i:
        valor = 'quesos'
    elif 'Cebolla' in i:
        valor = 'cebolla'
    elif 'Ajos' in i:
        valor = 'ajo'
    else:
        valor = np.nan
            
    return valor
df_bonarea['tipo'] = df_bonarea.caracteristicas.apply(tipo_producto_bonarea)


In [13]:
results_mask_bonarea = df_bonarea[df_bonarea['tipo'].isnull()] #Comprovar que tipos son nulos para ver si hay algun error.


In [14]:
df_bonarea = df_bonarea.loc[~df_bonarea['caracteristicas'].str.contains('Salsa')] 
#Observo que las salsa estan en tipo sal, como no me interesan. Las dropeo

In [15]:
df_bonarea.dropna(inplace=True)


C:\Users\manya\AppData\Local\Temp\ipykernel_30864\1884392361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bonarea.dropna(inplace=True)


#### Dia

In [16]:
df_dia = df_supermercados(ingredientes_dia)

In [17]:
def tipo_producto_dia (i,j): #a la función le paso dos valores que seran los de la columna link y la columna caracteristicas
    valor = 0
        
    if 'tomate' in i:
        valor ='tomate'
    elif 'sal' in i: 
        valor = 'sal'
    elif 'aceite' in i:
        valor = 'aceite' 
    elif 'leche' in i:
        valor = 'leche'
    elif 'mixto' in i and 'picada' in j: 
        valor = 'preparado mixto carne'
    elif 'canelones' in j or 'cannelloni' in j:
        valor = 'pasta'
    elif 'mantequilla' in i:
        valor = 'mantequilla'
    elif 'harina' in j: #columna caracteristicas
        valor ='harina'
    elif 'queso' in i:
        valor = 'quesos'
    elif 'cebolla' in i and 'Cebolla' in j: # en el link no puedo diferenciar, por eso uso column caracterisiticas
        valor = 'cebolla'
    elif 'ajo' in i and 'Ajo' in j:
        valor = 'ajo'
    else:
        valor = np.nan
            
    return valor

df_dia['tipo'] = df_dia.apply(lambda x: tipo_producto_dia(x['link'], x['caracteristicas']), axis=1)

In [18]:
#Compruebo que me quedan de valores nulos en tipo = esta ok ya puedo dropear los valores nulos
result_mask_dia = df_dia[df_dia['tipo'].isnull()]
df_dia.dropna(inplace = True) 

#### Bonpreu

In [49]:
df_bonpreu = df_supermercados (ingredientes_bonpreu)


In [50]:
def tipo_producto_bonpreu (i): 
    valor = 0
        
    if 'Tomàquet' in i:
        valor ='tomate'
    elif 'Sal' in i: 
        valor = 'sal'
    elif "Oli d'oliva" in i:
        valor = 'aceite' 
    elif 'Llet' in i:
        valor = 'leche'
    elif 'Preparat' in i or 'porc i vedella' in i: 
        valor = 'preparado mixto carne'
    elif 'Canelons' in i or 'canelons' in i:
        valor = 'pasta'
    elif 'Mantega' in i:
        valor = 'mantequilla'
    elif 'Farina' in i:
        valor ='harina'
    elif 'Formatge' in i:
        valor = 'quesos'
    elif 'Ceba' in i: 
        valor = 'cebolla'
    elif 'Alls secs' in i:
        valor = 'ajo'
    else:
        valor = np.nan
            
    return valor

df_bonpreu['tipo'] = df_bonpreu.caracteristicas.apply(tipo_producto_bonpreu)

In [93]:
#Compruebo que valores son nan y si me interesan. No me interesan. Los borro
results_mask_bonpreu = df_bonpreu[df_bonpreu['tipo'].isnull()]
df_bonpreu.dropna(inplace=True)
df_bonpreu

,precio_unidad,caracteristicas,precio_kg_l,cantidad_producto,link,tipo
0,1.1,ASTURIANA Llet sencera en cartró,1.12,0.982,https://www.compraonline.bonpreuesclat.cat//pr...,leche
1,6.7,ASTURIANA Llet sencera 6x1L en cartró,1.12,5.982,https://www.compraonline.bonpreuesclat.cat//pr...,leche
2,0.9,BONPREU Llet sencera en cartró,0.91,0.989,https://www.compraonline.bonpreuesclat.cat//pr...,leche
3,1.2,LLET NOSTRA Llet sencera en cartró,1.24,0.968,https://www.compraonline.bonpreuesclat.cat//pr...,leche
4,1.2,TERRA I TAST Llet sencera Km0 en ampolla,1.25,0.960,https://www.compraonline.bonpreuesclat.cat//pr...,leche
...,...,...,...,...,...,...
187,15.0,BORGES Oli d'oliva verge extra,7.65,1.961,https://www.compraonline.bonpreuesclat.cat//pr...,aceite
189,24.0,COOSUR Oli d'oliva suau,4.80,5.000,https://www.compraonline.bonpreuesclat.cat//pr...,aceite
190,7.9,BORGES Oli d'oliva verge extra,7.99,0.989,https://www.compraonline.bonpreuesclat.cat//pr...,aceite
191,4.5,BONPREU Oli d'oliva verge extra eco,9.18,0.490,https://www.compraonline.bonpreuesclat.cat//pr...,aceite


### Diccionario recetas y funciones comunes para crear columnas en dfs

In [66]:
def cantidad_receta (df):
    df['cantidad_receta'] = df['tipo'].map(dict_receta)
    
    return df

In [67]:
def unidades_comprar (df):
    df["unidades"] = df["cantidad_receta"] / df["cantidad_producto"]
    
    return df
    

In [68]:
def redondear_unidades(x):
    entero = int(x)
    if x == entero:
        return entero
    elif x != entero:
        return entero + 1

In [69]:
def coste (df):
    df['coste'] = round(df["precio_unidad"] * df["unidades"],1)
    
    return df

In [70]:
##Bonarea
df_bonarea1 = df_bonarea.copy()
cantidad_receta(df_bonarea1)
unidades_comprar (df_bonarea1)
df_bonarea1['unidades'] = df_bonarea1['unidades'].apply(redondear_unidades)
coste (df_bonarea1)

,precio_unidad,caracteristicas,precio_kg_l,cantidad_producto,tipo,cantidad_receta,unidades,coste
0,0.55,Tomate frito bric400 g,1.38,0.4,tomate,0.10,1,0.6
2,1.34,Tomate frito con cebolla570 g,2.35,0.6,tomate,0.10,1,1.3
14,0.80,Tomate frito Dantza390 g,2.05,0.4,tomate,0.10,1,0.8
15,0.96,Tomate frito Fruco380 g,2.53,0.4,tomate,0.10,1,1.0
16,2.18,Tomate frito Helios con aceite de oliva560 g,3.89,0.6,tomate,0.10,1,2.2
...,...,...,...,...,...,...,...,...
265,1.70,Queso rallado emmental200 g,8.50,0.2,quesos,0.25,2,3.4
266,1.80,Queso rallado 4 quesos200 g,9.00,0.2,quesos,0.25,2,3.6
267,1.39,Queso rallado Grana Padanoaprox. 100 g,13.90,0.1,quesos,0.25,3,4.2
268,0.75,Queso en polvo Hochland parma estilo italiano50 g,15.00,0.1,quesos,0.25,3,2.2


In [71]:
##DIA
df_dia1 = df_dia.copy()
cantidad_receta(df_dia1)
unidades_comprar (df_dia1)
df_dia1['unidades'] = df_dia1['unidades'].apply(redondear_unidades)
coste (df_dia1)

,precio_unidad,caracteristicas,precio_kg_l,cantidad_producto,link,tipo,cantidad_receta,unidades,coste
0,0.30,SAL DIA de mesa fina paquete 1 Kg,0.30,1.0,https://www.dia.es//compra-online/despensa/sal...,sal,0.015,1,0.3
1,0.30,SAL DIA marina yodada paquete 1 Kg,0.30,1.0,https://www.dia.es//compra-online/despensa/sal...,sal,0.015,1,0.3
2,0.30,SAL DIA marina gruesa paquete 1 Kg,0.30,1.0,https://www.dia.es//compra-online/despensa/sal...,sal,0.015,1,0.3
3,0.85,COSTA sal yodo+flúor paquete 1 Kg,0.85,1.0,https://www.dia.es//compra-online/despensa/sal...,sal,0.015,1,0.8
4,1.24,COSTA sal plus+ paquete 1 Kg,1.24,1.0,https://www.dia.es//compra-online/despensa/sal...,sal,0.015,1,1.2
...,...,...,...,...,...,...,...,...,...
131,3.45,SOLIS tomate frito frasco 2 x 460 gr,3.75,0.9,https://www.dia.es//compra-online/despensa/sal...,tomate,0.100,1,3.4
132,1.90,HELIOS tomate frito frasco 570 gr,3.33,0.6,https://www.dia.es//compra-online/despensa/sal...,tomate,0.100,1,1.9
133,1.99,SOLIS tomate frito estilo casero lata pack 3x...,6.63,0.3,https://www.dia.es//compra-online/despensa/sal...,tomate,0.100,1,2.0
134,1.85,HIDA Pisto sofrito de verduras casero lata 340 gr,5.44,0.3,https://www.dia.es//compra-online/despensa/sal...,tomate,0.100,1,1.8


In [72]:
##Bonpreu
df_bonpreu1 = df_bonpreu.copy()
cantidad_receta(df_bonpreu1)
unidades_comprar (df_bonpreu1)
df_bonpreu1['unidades'] = df_bonpreu1['unidades'].apply(redondear_unidades)
coste (df_bonpreu1)

,precio_unidad,caracteristicas,precio_kg_l,cantidad_producto,link,tipo,cantidad_receta,unidades,coste
0,1.1,ASTURIANA Llet sencera en cartró,1.12,0.982,https://www.compraonline.bonpreuesclat.cat//pr...,leche,1.00,2,2.2
1,6.7,ASTURIANA Llet sencera 6x1L en cartró,1.12,5.982,https://www.compraonline.bonpreuesclat.cat//pr...,leche,1.00,1,6.7
2,0.9,BONPREU Llet sencera en cartró,0.91,0.989,https://www.compraonline.bonpreuesclat.cat//pr...,leche,1.00,2,1.8
3,1.2,LLET NOSTRA Llet sencera en cartró,1.24,0.968,https://www.compraonline.bonpreuesclat.cat//pr...,leche,1.00,2,2.4
4,1.2,TERRA I TAST Llet sencera Km0 en ampolla,1.25,0.960,https://www.compraonline.bonpreuesclat.cat//pr...,leche,1.00,2,2.4
...,...,...,...,...,...,...,...,...,...
187,15.0,BORGES Oli d'oliva verge extra,7.65,1.961,https://www.compraonline.bonpreuesclat.cat//pr...,aceite,0.06,1,15.0
189,24.0,COOSUR Oli d'oliva suau,4.80,5.000,https://www.compraonline.bonpreuesclat.cat//pr...,aceite,0.06,1,24.0
190,7.9,BORGES Oli d'oliva verge extra,7.99,0.989,https://www.compraonline.bonpreuesclat.cat//pr...,aceite,0.06,1,7.9
191,4.5,BONPREU Oli d'oliva verge extra eco,9.18,0.490,https://www.compraonline.bonpreuesclat.cat//pr...,aceite,0.06,1,4.5


### Aggregation

In [73]:
sub_dfdia = df_dia1.loc[df_dia1.groupby('tipo')['coste'].idxmin()]
sub_dfdia.loc['Total','coste'] = sub_dfdia['coste'].sum()
sub_dfdia

,precio_unidad,caracteristicas,precio_kg_l,cantidad_producto,link,tipo,cantidad_receta,unidades,coste
64,1.99,CAPRICHO ANDALUZ set aliño aceite de oliva y v...,1.99,1.0,https://www.dia.es//compra-online/despensa/ace...,aceite,0.060,1.0,2.0
23,0.39,Ajo unidad (62 gr aprox.),0.39,1.0,https://www.dia.es//compra-online/frescos/verd...,ajo,0.060,1.0,0.4
16,0.94,Cebolla dulce unidad (350 gr aprox.),2.69,0.3,https://www.dia.es//compra-online/frescos/verd...,cebolla,0.200,1.0,0.9
70,0.72,DIA harina de trigo paquete 1 Kg,0.72,1.0,https://www.dia.es//compra-online/despensa/pas...,harina,0.100,1.0,0.7
106,0.90,DIA LACTEA leche semidesnatada envase 1 lt,0.90,1.0,https://www.dia.es//compra-online/despensa/lac...,leche,1.000,1.0,0.9
28,2.15,DIA LACTEA mantequilla con sal barqueta 250 gr,8.60,0.2,https://www.dia.es//compra-online/despensa/lac...,mantequilla,0.100,1.0,2.2
80,1.25,EL PAVO cannelloni clásico caja 125 gr,10.00,0.1,https://www.dia.es//compra-online/despensa/pas...,pasta,0.250,3.0,3.8
83,3.99,SELECCIÓN DE DIA preparado de carne picada de ...,7.98,0.5,https://www.dia.es//compra-online/frescos/carn...,preparado mixto carne,0.500,1.0,4.0
34,1.53,DIA LACTEA queso especial para gratinar rallad...,7.65,0.2,https://www.dia.es//compra-online/frescos/char...,quesos,0.250,2.0,3.1
0,0.30,SAL DIA de mesa fina paquete 1 Kg,0.30,1.0,https://www.dia.es//compra-online/despensa/sal...,sal,0.015,1.0,0.3


In [76]:
sub_dfbonarea = df_bonarea1.loc[df_bonarea1.groupby('tipo')['coste'].idxmin()]
sub_dfbonarea.loc['Total','coste'] = sub_dfbonarea['coste'].sum()
sub_dfbonarea

,precio_unidad,caracteristicas,precio_kg_l,cantidad_producto,tipo,cantidad_receta,unidades,coste
143,2.35,Aceite de oliva virgen extra Borges arbequina2...,9.40,0.2,aceite,0.060,1.0,2.4
163,1.32,Ajos secos malla250 g,5.28,0.2,ajo,0.060,1.0,1.3
177,1.91,Cebolla tierna400 g,4.78,0.4,cebolla,0.200,1.0,1.9
125,0.72,Harina de trigo1 kg,0.72,1.0,harina,0.100,1.0,0.7
52,1.00,Leche desnatada calcio brick1 l,1.00,1.0,leche,1.000,1.0,1.0
30,1.65,Mantequilla Ato125 g,13.20,0.1,mantequilla,0.100,1.0,1.6
81,0.94,Canelones El Pavo 20 u.125 g,7.52,0.1,pasta,0.250,3.0,2.8
230,6.78,Carne picada de vacunoaprox. 649 g,10.45,0.6,preparado mixto carne,0.500,1.0,6.8
261,0.72,Queso rallado fundido100 g,7.20,0.1,quesos,0.250,3.0,2.2
128,0.30,Sal fina1 kg,0.30,1.0,sal,0.015,1.0,0.3


In [81]:
sub_dfbonpreu = df_bonpreu1.loc[df_bonpreu1.groupby('tipo')['coste'].idxmin()]
sub_dfbonpreu.loc['Total','coste'] = sub_dfbonpreu['coste'].sum()
sub_dfbonpreu

,precio_unidad,caracteristicas,precio_kg_l,cantidad_producto,link,tipo,cantidad_receta,unidades,coste
185,2.8,CARBONELL Oli d'oliva verge extra Amanides,14.45,0.194,https://www.compraonline.bonpreuesclat.cat//pr...,aceite,0.060,1.0,2.8
63,0.3,Alls secs 1 u.,4.89,0.061,https://www.compraonline.bonpreuesclat.cat//pr...,ajo,0.060,1.0,0.3
65,0.3,Ceba vermella 1 u.,1.45,0.207,https://www.compraonline.bonpreuesclat.cat//pr...,cebolla,0.200,1.0,0.3
132,0.7,BONPREU Farina de blat,0.70,1.000,https://www.compraonline.bonpreuesclat.cat//pr...,harina,0.100,1.0,0.7
6,1.2,ATO Llet sencera en cartró,1.20,1.000,https://www.compraonline.bonpreuesclat.cat//pr...,leche,1.000,1.0,1.2
38,2.2,ARIAS Mantega micro-pastilla,19.08,0.115,https://www.compraonline.bonpreuesclat.cat//pr...,mantequilla,0.100,1.0,2.2
129,1.1,EL PAVO Plaques per canelons,0.06,18.333,https://www.compraonline.bonpreuesclat.cat//pr...,pasta,0.250,1.0,1.1
76,4.9,BONPREU Burger meat de porc i vedella,8.32,0.589,https://www.compraonline.bonpreuesclat.cat//pr...,preparado mixto carne,0.500,1.0,4.9
82,0.9,BONPREU Formatge emmental ratllat,9.90,0.091,https://www.compraonline.bonpreuesclat.cat//pr...,quesos,0.250,3.0,2.7
146,0.3,BONPREU Sal marina fina,0.30,1.000,https://www.compraonline.bonpreuesclat.cat//pr...,sal,0.015,1.0,0.3
